# Manipulación Avanzada de Datos
------------------------

En esta sección aprenderemos a manipular nuestro dataframe haciendo agrupaciones de datos o trabajando con más de un dataframe a la vez


## 1. Sumarización de Datos

Las sentencias de agrupamiento de datos nos ayudan a brindar información resumida que pueda ser facilmente analizada por diversas personas.

<img src='./img/group_by.jpg'>

El agrupamiento de datos implica utilizar funciones de agregacion como: `count`, `sum`, `mean`, `min`, `max` a una columna del df

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_reviews = pd.read_csv('./src/winemag-data-130k-v2.csv')
df_reviews.head(2)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [9]:
df_reviews_group = df_reviews.groupby(['country']).price.agg(['count', 'mean', 'min', 'max'])

df_reviews_group.head()

,count,mean,min,max
country,,,,
Argentina,1887,23.604663,4.0,230.0
Armenia,1,14.000000,14.0,14.0
Australia,1158,35.786701,6.0,850.0
Austria,1364,30.846774,7.0,150.0
Bosnia and Herzegovina,1,13.000000,13.0,13.0


In [14]:
condition = df_reviews.country == 'Peru'
df_reviews[condition].groupby(['country', 'province']).agg({
    'price':['mean', 'min', 'max'],
    'points': 'mean',
    'country':'count'
    })

price             points country
                  mean   min   max   mean   count
country province                                 
Peru    Ica       20.5  10.0  68.0   84.0       8

In [12]:
df_reviews_group.sort_values(by='mean', ascending=False).head(5)

,count,mean,min,max
country,,,,
Switzerland,4,97.000000,30.0,160.0
England,33,51.636364,25.0,80.0
Hungary,61,50.819672,10.0,764.0
France,8961,41.577949,5.0,2500.0
Germany,1030,40.592233,5.0,775.0


## 2. Manipulando más de un DataFrame

### 2.1 Unificando o Concatenando DataFrames

Esto nos permite unificar información de Dataframs cuyas columnas sean iguales

<img src='https://pandas.pydata.org/docs/_images/08_concat_row.svg'>

In [15]:
# A manera de ejemplo veremos la unificación de 2 df's cuya data es similar

import pandas as pd

columns = ["date.utc", "location", "parameter", "value"]

df_air_quality_no2 = pd.read_csv("./src/air_quality_no2_long.csv", parse_dates=True, usecols=columns)
df_air_quality_pm25 = pd.read_csv("./src/air_quality_pm25_long.csv", parse_dates=True, usecols=columns)


In [16]:
df_air_quality_no2.head(2)



,date.utc,location,parameter,value
0,2019-06-21 00:00:00+00:00,FR04014,no2,20.0
1,2019-06-20 23:00:00+00:00,FR04014,no2,21.8


In [17]:
df_air_quality_pm25.head()

,date.utc,location,parameter,value
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5


In [23]:
# Unificando la información en un único df

df_air_quality = pd.concat([df_air_quality_pm25, df_air_quality_no2], axis=0)
df_air_quality.head()

,date.utc,location,parameter,value
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5


In [24]:
print('Shape of the ``air_quality_pm25`` table: ', df_air_quality_pm25.shape)

print('Shape of the ``air_quality_no2`` table: ', df_air_quality_no2.shape)

print('Shape of the resulting ``air_quality`` table: ', df_air_quality.shape)

Shape of the ``air_quality_pm25`` table:  (1110, 4)
Shape of the ``air_quality_no2`` table:  (2068, 4)
Shape of the resulting ``air_quality`` table:  (3178, 4)


### 2.2 Joins

Podemos unir dos dataframes en funcion de sus columnas comunes usando `merge`

La operacion merge implica combinar 2 df a partir de uno o más valores llave o `key`

<img src='./img/merge.png'>

In [20]:
# Unificaremos la información consolidada del df previo 
df_stations_coord = pd.read_csv("./src/air_quality_stations.csv")
df_stations_coord.head(2)

,location,coordinates.latitude,coordinates.longitude
0,BELAL01,51.23619,4.38522
1,BELHB23,51.17030,4.34100


In [21]:
# como llame emplearemos la columna 'location'

df_air_quality = pd.merge(df_air_quality, df_stations_coord, how="inner", on="location")
df_air_quality.head()

,date.utc,location,parameter,value,coordinates.latitude,coordinates.longitude
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0,51.20966,4.43182
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5,51.20966,4.43182
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5,51.20966,4.43182
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0,51.20966,4.43182
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5,51.20966,4.43182


In [25]:
# como llame emplearemos la columna 'location'


df_air_quality = pd.merge(df_air_quality, df_stations_coord, how="inner", left_on='location', right_on='location')
df_air_quality.head()

,date.utc,location,parameter,value,coordinates.latitude,coordinates.longitude
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0,51.20966,4.43182
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5,51.20966,4.43182
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5,51.20966,4.43182
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0,51.20966,4.43182
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5,51.20966,4.43182


Como punto general existen diferentes formas de combinar los dataframe, siendo el método `inner` el utilizado por defecto

<img src='./img/merge_tipos.png'>

## Información Adicional


- Group By Explicado:  https://learnsql.com/blog/group-by-in-sql-explained/

- Combinando Múltiples dataFrames : https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html#min-tut-08-combine

In [26]:
# 

# 

import pandas as pd
pd.set_option('display.max_columns', None)

df = pd.read_excel('./src/reactiva.xlsx',sheet_name='TRANSFERENCIAS 2020')

df.head(2)


,ID,CODIGO PAIS,CODIGO ENTIDAD,UBIGEO,SNIP,CUI,REGION,PROVINCIA,DISTRITO,PROYECTO,DISPOSITIVO LEGAL,AMBITO,UNIDAD EJECUTORA,TOTAL EMPLEOS,POBLACION BENEFICIARIA,TIPOLOGIA,TIPO MONEDA,MONTO DE INVERSIÓN,TIPO MONEDA.1,MONTO DE TRANSFERENCIA 2020,ESTADO
0,1,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,"DU N° 070-2020,",URBANO,MUNICIPALIDAD DISTRITAL DE QUICHUAY,121,1529,Pista y Vereda,PEN,4105693.00,PEN,4105693,En Ejecución
1,2,PE,11476,90612,213528,2215425,HUANCAVELICA,HUAYTARA,SAN ISIDRO,MEJORAMIENTO VIAL DEL CERCADO DE LA LOCALIDAD ...,"DU N° 070-2020,",RURAL,MUNICIPALIDAD DISTRITAL DE SAN ISIDRO - HUIRPA...,63,648,Pista y Vereda,PEN,2120069.47,PEN,413572,En Ejecución


In [30]:
df.columns

Index(['ID', 'CODIGO PAIS', 'CODIGO ENTIDAD', 'UBIGEO', 'SNIP', 'CUI',
       'REGION', 'PROVINCIA', 'DISTRITO', 'PROYECTO', 'DISPOSITIVO LEGAL',
       'AMBITO', 'UNIDAD EJECUTORA', 'TOTAL EMPLEOS', 'POBLACION BENEFICIARIA',
       'TIPOLOGIA', 'TIPO MONEDA', 'MONTO DE INVERSIÓN', 'TIPO MONEDA.1',
       'MONTO DE TRANSFERENCIA 2020', 'ESTADO'],
      dtype='object')

In [29]:
dicx ={
    'Moneda': ['PEN', 'USD', 'EUR'],
    'Descripcion': ['Soles Peruanos', 'Dolar Americano', 'Euro']
}

df_moneda = pd.DataFrame(dicx)
df_moneda

,Moneda,Descripcion
0,PEN,Soles Peruanos
1,USD,Dolar Americano
2,EUR,Euro


In [32]:
df_merge = pd.merge(df, df_moneda, left_on='TIPO MONEDA', right_on='Moneda')
df_merge.head(1)

,ID,CODIGO PAIS,CODIGO ENTIDAD,UBIGEO,SNIP,CUI,REGION,PROVINCIA,DISTRITO,PROYECTO,DISPOSITIVO LEGAL,AMBITO,UNIDAD EJECUTORA,TOTAL EMPLEOS,POBLACION BENEFICIARIA,TIPOLOGIA,TIPO MONEDA,MONTO DE INVERSIÓN,TIPO MONEDA.1,MONTO DE TRANSFERENCIA 2020,ESTADO,Moneda,Descripcion
0,1,PE,11476,120127,346548,2307983,JUNIN,HUANCAYO,QUICHUAY,MEJORAMIENTO DE PISTAS Y VEREDAS EN LA CA. LIM...,"DU N° 070-2020,",URBANO,MUNICIPALIDAD DISTRITAL DE QUICHUAY,121,1529,Pista y Vereda,PEN,4105693.0,PEN,4105693,En Ejecución,PEN,Soles Peruanos


In [ ]:
# Eliminar columnas innecesarias
# Renombrar columnas
# Generame un reporte excel para cada uno de las regiones, Almacena cada dato en un archivo excel diferente
# Envia un correo a cada gobernador según el listado de region:correo

In [33]:
df_merge.REGION.unique()

array(['JUNIN', 'HUANCAVELICA', 'CAJAMARCA', 'LIMA', 'APURIMAC', 'PASCO',
       'AYACUCHO', 'ICA', 'SAN MARTIN', 'TACNA', 'PUNO', 'PIURA',
       'LA LIBERTAD', 'CUSCO', 'AMAZONAS', 'UCAYALI', 'HUANUCO', 'CALLAO',
       'AREQUIPA', 'MOQUEGUA', 'TUMBES', 'LAMBAYEQUE', 'ANCASH', 'LORETO'],
      dtype=object)

In [36]:
for region in df_merge.REGION.unique():

    condition = df_merge.REGION == region
    
    nombre_excel = f'./output/reportes/Reporte-{region}.xlsx'
    df_merge[condition].to_excel(nombre_excel, sheet_name='ReporteRegion', index=False)
    print(f'Se genero reporte Region {region}')

Se genero reporte Region JUNIN
Se genero reporte Region HUANCAVELICA
Se genero reporte Region CAJAMARCA
Se genero reporte Region LIMA
Se genero reporte Region APURIMAC
Se genero reporte Region PASCO
Se genero reporte Region AYACUCHO
Se genero reporte Region ICA
Se genero reporte Region SAN MARTIN
Se genero reporte Region TACNA
Se genero reporte Region PUNO
Se genero reporte Region PIURA
Se genero reporte Region LA LIBERTAD
Se genero reporte Region CUSCO
Se genero reporte Region AMAZONAS
Se genero reporte Region UCAYALI
Se genero reporte Region HUANUCO
Se genero reporte Region CALLAO
Se genero reporte Region AREQUIPA
Se genero reporte Region MOQUEGUA
Se genero reporte Region TUMBES
Se genero reporte Region LAMBAYEQUE
Se genero reporte Region ANCASH
Se genero reporte Region LORETO
